In [4]:
import numpy as np
import matplotlib.pyplot as plt

def empuxo(RPM, d,pitch, V0, numero_de_motores):
# O empuxo de um propeller dado um motor que fornece uma rotacação RPM
# V0 = propeller forward airspeed , velocidade da hélice para a frente 

    return ((4.392399/10**(8))*RPM*((d**(3.5))/np.sqrt(pitch))*(4.23333*10**(-4)*RPM*pitch - V0))*numero_de_motores

def empuxo_dada_a_tensao(V,KV, d,pitch, V0, numero_de_motores):
    RPM = V*KV
    return ((4.392399/10**(8))*RPM*((d**(3.5))/np.sqrt(pitch))*(4.23333*10**(-4)*RPM*pitch - V0))*numero_de_motores

def V0_dado_um_teste_de_fabricante(F,d, pitch, v, KV):
    RPM = KV*v
    #-V0 = (F/((4.392399*10**-8)*RPM*(d**3.5)/np.sqrt(pitch))) - 4.2*(10**-4)*RPM*pitch
    V0 = -(F/((4.392399*10**-8)*RPM*(d**3.5)/np.sqrt(pitch))) + 4.2*(10**-4)*RPM*pitch
    return V0

# ============== Dimensionamento =========================================
# 1) diâmetro_max_propeller = aplicação(massa, tamanho do frame, taxa Empuxo-peso)
# 2)
def RPM(P,trust_to_weight_ratio , d,pitch, V0, numero_de_motores):
    F = P*trust_to_weight_ratio
    a = (4.392399/10**(8))*((d**(3.5))/np.sqrt(pitch))*(4.23333*10**(-4))*pitch
    b = -(4.392399/10**(8))*((d**(3.5))/np.sqrt(pitch))*V0
    c = - F/numero_de_motores
    # 0 = a RPM²   - b RPM  - c
    DELTA = b**(2) - 4*a*c
    RPM1 = (-b + np.sqrt(DELTA))/(2*a)
    #RPM2 = (-b - np.sqrt(DELTA))/(2*a)
    #print(RPM1, RPM2)
    return RPM1
# 3) Escolha da tensão -> numero de celulas

# 4)
def motor(n_celulas,RPM):
    V = 3.7
    if(n_celulas == 1):
        V = 3.7
    if(n_celulas == 2):
         V = 2*V
    if(n_celulas == 3):
        V = 3*V 
    if(n_celulas == 4):
        V = 4*V
        
    KV = RPM/V
    return KV

# 5) i = pesquisa_na_net(KV_MOTOR)
#6 ESC(V_motor, i_motor)
#7) autonomia = Escolhemos uma aunotomia
#8) 


def Bateria(v,autonomia_em_min,i_usual, imax, numero_de_motores):
    autonomia_em_h = autonomia_em_min/60
    i_usual_total = i_usual*numero_de_motores
    carga = autonomia_em_h*i_usual_total*1000
    C = (imax*numero_de_motores)/(carga/1000)
    # print(" Carga = " +str(carga)+ " mAh " " C = " +str(C))
    resposta = [carga, C]
    return resposta

    


# Calculos iniciais


In [5]:
g = 10
P = 1.7*g
E = empuxo(14208,9.4,5,6,4) 
trust_to_weight_ratio = E/(P) 
print(trust_to_weight_ratio) 

4.025769458447914


In [6]:
RPM(17,4.025769,9.4,5,6,4)

14207.999280651986

In [7]:
RPM(17,4.025769,9.4,5,6,4)

14207.999280651986

In [8]:
motor(4, 14207.999)

959.9999324324324

In [9]:
empuxo_dada_a_tensao(14.8,1290,10,4.7,6,4)

156.5069111957366

In [10]:
empuxo_dada_a_tensao(14.8,1200,10,6,6,4)

157.55093136440254

In [11]:
empuxo_dada_a_tensao(14.8,1200,10,6,6,4)

157.55093136440254

In [12]:
Bateria(14.8,12,15/4,20,4)


[3000.0, 26.666666666666668]

In [13]:
Bateria(14.8,20,20/4,20,4)

[6666.666666666666, 12.000000000000002]

In [14]:
Bateria(11.1,15,20/4,10,4)

[5000.0, 8.0]

In [15]:
v0 = V0_dado_um_teste_de_fabricante(10*1065/1000,10, 4.7, 11.1, 1290)
empuxo = empuxo_dada_a_tensao(14.8,1290, 10,4.7, v0, 4)
print(empuxo)
m_cm_taxa_4 = (empuxo/4)/10
print(m_cm_taxa_4)
RPM(m_cm_taxa_4*10,4, 10,4.7, v0, 4)

104.3635570159809
2.6090889253995226


19091.999999999996

# Requisitos de Projeto

## 1) Caluclo do RPM dada um propeller

In [16]:
#  RPM(P,trust_to_weight_ratio , d,pitch, V0, numero_de_motores):
m_bateria_inicial = 554
m_drone = 929 + m_bateria_inicial + 14.5 + 2.7 + 2 + ((105 + 190)/2) + 10 + 250 + 90
m_drone = m_drone/1000
print("Masssa drone = " +str(m_drone)+ " kg ")
print()
g = 10
# valroes uteis:
P_drone = m_drone*g
trust_to_weight_ratio = 4 
V0 = 6
numero_de_motores = 4

d = [9,10,10,8,10,9] 
pitch = [6,5.5,4.5,4.5,4.5,4.7]
rpm = []

# for i in range(len(pitch)):
for i in range(len(pitch)):
    rpm.append(RPM(P,trust_to_weight_ratio , d[i], pitch[i], V0, numero_de_motores))
    print("Velocidade ang =" +str(round(rpm[i])) + " rpm" + " -- opção " + str(i))   
#numero = np.arange(0, len(pitch), 1)
#plt.plot(numero,rpm,color = 'red',linewidth=4,label="Circuito Completo")



Masssa drone = 1.9997 kg 

Velocidade ang =14298.0 rpm -- opção 0
Velocidade ang =12466.0 rpm -- opção 1
Velocidade ang =13355.0 rpm -- opção 2
Velocidade ang =18898.0 rpm -- opção 3
Velocidade ang =13355.0 rpm -- opção 4
Velocidade ang =15476.0 rpm -- opção 5


## 2) Escolha uma tensão
Escolhemos 3s ou 4s , isto é, 11.1 ou 14.8

## 3) Definindo-se o motor pelo KV

In [17]:
# Motor 3S
print("Bateriu 3S")
print()
KV = []
for i in range(len(pitch)):
    KV.append(motor(3,rpm[i]))
    print("KV =" +str(round(KV[i])) + " -- opção " + str(i) ) 
# 4S
print()
print("Bateriu 4S")
print()
KV = []
for i in range(len(pitch)):
    KV.append(motor(4,rpm[i]))
    print("KV =" +str(round(KV[i])) + " -- opção " + str(i) ) 
print()    
print("Comprar com folga pra cima")

Bateriu 3S

KV =1288.0 -- opção 0
KV =1123.0 -- opção 1
KV =1203.0 -- opção 2
KV =1703.0 -- opção 3
KV =1203.0 -- opção 4
KV =1394.0 -- opção 5

Bateriu 4S

KV =966.0 -- opção 0
KV =842.0 -- opção 1
KV =902.0 -- opção 2
KV =1277.0 -- opção 3
KV =902.0 -- opção 4
KV =1046.0 -- opção 5

Comprar com folga pra cima


### 5) i = pesquisa_na_net(KV_MOTOR)
[Motor 1](https://www.banggood.com/pt/Racerstar-BR2826-1290KV-2-4S-Brushless-Motor-For-RC-Models-p-1114330.html?rmmds=detail-bottom-alsobought__5&cur_warehouse=CN)

[Motor 2](https://www.banggood.com/pt/Flash-Hobby-D3542-1000KV-1250KV-1450KV-2-4S-Brushless-Motor-For-RC-Airplane-p-1628286.html?rmmds=detail-bottom-alsobought__8&ID=517699&cur_warehouse=CN)

[Motor 3](https://www.banggood.com/pt/Racerstar-BR3536-1200KV-2-4S-Brushless-Motor-For-FPV-RC-Airplane-Model-p-1384219.html?rmmds=detail-left-hotproducts__5&cur_warehouse=USA)

[Motor 4](https://www.rcpapa.com/collections/drone-motors/products/t-motor-f60-pro-iv-1950-2552kv-fpv-quad-motor)

[Motor 5](https://www.banggood.com/pt/Racerstar-BR2826-1290KV-2-4S-Brushless-Motor-For-RC-Models-p-1114330.html?rmmds=detail-bottom-alsobought__5&cur_warehouse=CN)

[Motor 6](https://www.racerstar.com/racerstar-br3536-1200kv-2-4s-brushless-motor-for-fpv-rc-airplane-model-p-298.html)

[Motor 7](https://www.racerstar.com/racerstar-br2814-1400kv-3-4s-brushless-motor-for-rc-airplane-model-p-291.html)


In [18]:
numero     = [  1 ,    2,   3,    4,    5,    6,   7,    8,    9,   10,    11,  12]
Motor      = [1.1 ,  1.2, 2.1, 2.2 , 2.3 , 2.4 ,3.1 , 3.2 , 5.1 ,5.2  , 7.1  ,7.2  ]
propellers = [1045, 9045,1250, 1170, 8070, 1060,1150, 1060, 9050, 1045, 9060 ,8050]
tensoes    = [11.1, 11.1, 7.4, 14.8, 4.7 , 7.4 ,11.1, 11.1, 11.1, 14.8, 11.1 ,14.8 ]
corrente   = [16.6, 13.8,  80, 46.5, 90  , 74  , 26 , 35  , 21.5, 29  , 47.5 ,58   ]
massa_g   = [1065, 800 ,1420, 1420, 1350, 1260,1360, 1580, 1130, 1310, 1950 ,2300]


Empuxo_taxa_3 = np.dot(4/1000,massa_g)
print(Empuxo_taxa_3)

Empuxo_taxa_4 = np.dot(4/1000,massa_g)
print(Empuxo_taxa_4)


[4.26 3.2  5.68 5.68 5.4  5.04 5.44 6.32 4.52 5.24 7.8  9.2 ]
[4.26 3.2  5.68 5.68 5.4  5.04 5.44 6.32 4.52 5.24 7.8  9.2 ]


### 6 ESC(V_motor, i_motor)
### 7) autonomia = Escolhemos uma aunotomia
queremos 12 min
### 8) Bateria

In [19]:
autonomia_em_min = 12
# i_usual = "fudeu"
i_usual = np.dot((1/4),imax)
v = [11.1, 11.1, 7.4, 14.8, 4.7 , 7.4 ,11.1, 11.1, 11.1, 14.8, 11.1,14.8 ]
imax = [16.6, 13.8,  80, 46.5, 90  , 74  , 26 , 35  , 21.5, 29  , 47.5,58 ]
numero_de_motores = 4
valores = []
for i in range(len(v)):
    valores.append(Bateria(v[i],autonomia_em_min,i_usual[i], imax[i], numero_de_motores))
    print("Carga =" + str(round(valores[i][0]))+ "mAh /" +"C = "+ str(round(valores[i][1])) +"/ V = " + str(v[i])+" v /" +"kg_3 = "+ str(round(Empuxo_taxa_3[i])) +" / kg_4 =  "+str(round(Empuxo_taxa_4[i])) + " -- opção " + str(i)) 
    
    

Carga =4300.0mAh /C = 15.0/ V = 11.1 v /kg_3 = 4.0 / kg_4 =  4.0 -- opção 0


IndexError: index 1 is out of bounds for axis 0 with size 1

### 9) Estimativa da estimativa do consumo da eletrônica


In [20]:
Bateria_atual = 6000
consumo_da_eletronica = Bateria_atual - Bateria(14.8,20,15/4, 20, 4)[0]
print("No drone SKY3100 = " + str(consumo_da_eletronica) + "mAh")

No drone SKY3100 = 1000.0mAh


Com os upgrades:

In [21]:
t = 12/60
jetson_nano = 2*t
odroid = 4*t 
lidar_rotativo = 2*t
camera_termica = 0.120*t 
upgrades_carga_consumida = jetson_nano + odroid + lidar_rotativo + camera_termica
print("upgrades = " + str(upgrades_carga_consumida) + "mAh")

upgrades = 1.624mAh


Consumo tota da eletronica = 1000mAh

# Drone Swarm
* Frame 300 mm
* Massa caixa = 1kg

1) Motor Normal
* 2 Blade
* Max prop size= 8"
* Massa drone = 1.230 Kg

* Numero de drones = 2
* Massa suportada por drone = 1.73 kg
* thrust_to_weight_ratio = 2.9
* Precisa levantar = 5 Kg
* Motor + Propeller = 1250g
* M_para-corrente-média = 500

2) Motor racer
* 3 Blade
* Max prop size= 8"
* Massa drone = 0.930 Kg

* Numero de drones = 2
* Massa suportada por drone = 1.43 kg
* thrust_to_weight_ratio = 2.9
* Precisa levantar = 4.2 Kg
* Motor + Propeller = 1050 g
* M_para-corrente-média = 420

In [41]:
autonomia_em_min = 7.9
6.5

# i_usual =. "fudeu"
i_usual = [7.6, 7.5] #np.dot((1/4),imax)
v = [14.8, 11.1]
imax = [21.5, 38]
numero_de_motores = 4
valores = []
for i in range(len(v)):
    valores.append(Bateria(v[i],autonomia_em_min,i_usual[i], imax[i], numero_de_motores))
    print("Carga =" + str(round(valores[i][0]))+ "mAh /" +"C = "+ str(round(valores[i][1])) +"/ V = " + str(v[i])+" v /" +"kg_3 = "+ str(round(Empuxo_taxa_3[i])) +" / kg_4 =  "+str(round(Empuxo_taxa_4[i])) + " -- opção " + str(i)) 
    

Carga =4003mAh /C = 21/ V = 14.8 v /kg_3 = 4.0 / kg_4 =  4.0 -- opção 0
Carga =3950mAh /C = 38/ V = 11.1 v /kg_3 = 3.0 / kg_4 =  3.0 -- opção 1
